## Advanced Prompt Engineering

### 1. Few shot Learning

소량의 학습만으로 새로운 작업을 학습하거나 일반화하는 것을 의미한다. 모델에 어떻게 반응하는지를 원하는지에 대한 예시를 제공하는 것으로, 이 기술은 전체 아키텍처를 건드리지 않고도 모델 맞춤화를 가능하게 한다.

In [2]:
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
import os

with open('../../config/api.key') as file :
    lines = file.readlines()
    api_key = lines[0].strip()
    serp_api_key = lines[1].strip()
    langsmith_api_key = lines[2].strip()

openai.api_key = api_key

In [6]:
system_message = """
당신은 AI 마케팅 도우미입니다. 사용자가 새로운 제품 이름에 대한 캐치프레이즈를 만들 수 있도록 도와줍니다.
주어진 제품명에 대해 다음 예시와 비슷한 홍보 문구를 만들어주세요.

Apple - Think Different
Samsung - Do What You Can’t
SK 텔레콤 - 또 다른 세상을 만날 땐 잠시 꺼두셔도 좋습니다.
Amazon - Earth’s Most Customer-Centric Company
Microsoft - Cloud First
Google - Don’t Be Evil

제품명 : 
"""

product_name = '서강대학교 경제대학'

In [7]:
client = OpenAI(api_key = openai.api_key)

response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': product_name},
    ]
)

In [8]:
print(response.choices[0].message.content)

서강대학교 경제대학 - 미래를 디자인하는 경제의 중심


대부분의 경우, fewshot learning은 fine tuning을 적절한 도구로 생각할 수 있는 극도로 특수한 시나리오에서도 모델을 맞춤화할 수 있을 만큼 강력하다는 점을 명심하자. 실제로 적절한 few shot learning은 fine tuning process만큼 효과적일 수 있다. 다른 예를 살펴보자. 감정 분석에 특화된 모델을 개발하고자 한다고 가정해 보자. 이를 위해 긍정 또는 부정 등 원하는 결과와 함께 다양한 감정을 가진 일련의 텍스트 예시를 제공한다. 이 예제 세트는 지도 학습 과업을 위한 작은 훈련 세트에 불과하며, fine tuning과 유일한 차이점은 모델의 parameter를 업데이트 하지 않는다는 점이다.

In [9]:
system_message = """
당신은 텍스트를 전문적으로 분석하는 분석가입니다. 
텍스트의 감정을 positive, negative 만으로 분류합니다.

원문이 한국어가 아닌 경우, 한국어로 번역을 하되 번역된 원문은 제공하지 마세요.

다음 텍스트들을 예시로 사용할 수 있습니다.

텍스트 : "I love this product! It's fantastic and works perfectly."
Positive

텍스트 : "I'm really disappointed with the quality of the food."
Negative

텍스트 : "This is the best day of my life."
Positive

텍스트 : "I can't stand the noise in this restaurant."
Negative

감정만을 구두점 없이 출력합니다.

텍스트 : 
"""

분류기를 테스트하기 위해 Kaggle에서 제공되는 영화 리뷰의 IMDB 데이터베이스를 사용한다.

In [10]:
df = pd.read_csv('../../data/IMDB Dataset.csv', encoding = 'utf-8')
df['sentiment'] = df['sentiment'].replace(
    {0: 'negative', 1: 'positive'}
)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
df = df.sample(n=10, random_state=42)

In [12]:
df

,review,sentiment
33553,I really liked this Summerslam due to the look...,positive
9427,Not many television shows appeal to quite as m...,positive
199,The film quickly gets to a major chase scene w...,negative
12447,Jane Austen would definitely approve of this o...,positive
39489,Expectations were somewhat high for me when I ...,negative
42724,I've watched this movie on a fairly regular ba...,positive
10822,For once a story of hope highlighted over the ...,positive
49498,"Okay, I didn't get the Purgatory thing the fir...",positive
4144,I was very disappointed with this series. It h...,negative
36958,The first 30 minutes of Tinseltown had my fing...,negative


In [13]:
def process_text(text) : 
    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = [
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': text},
        ]
    )
    return response.choices[0].message.content

df['predicted'] = df['review'].apply(process_text)

In [14]:
df

,review,sentiment,predicted
33553,I really liked this Summerslam due to the look...,positive,negative
9427,Not many television shows appeal to quite as m...,positive,Positive
199,The film quickly gets to a major chase scene w...,negative,Negative
12447,Jane Austen would definitely approve of this o...,positive,Positive
39489,Expectations were somewhat high for me when I ...,negative,Negative
42724,I've watched this movie on a fairly regular ba...,positive,Positive
10822,For once a story of hope highlighted over the ...,positive,Positive
49498,"Okay, I didn't get the Purgatory thing the fir...",positive,Positive
4144,I was very disappointed with this series. It h...,negative,Negative
36958,The first 30 minutes of Tinseltown had my fing...,negative,Negative


label과 predicted 열을 비교한 결과를 보면 알 수 있듯이, 모델을 fine tuning하지 않고도 모든 리뷰를 정확하게 분류할 수 있다. 이것은 모델 전문화 측면에서 few shot learning 기술을 통해 달성할 수 있는 것의 한 예일 뿐이다.

### 2. Chain of Thought(CoT)

사고 연쇄(Chain of Thought)는 중간 추론 단계를 통해 복잡한 추론을 가능하게 하는 기법이다. 또한 모델이 자신의 추론을 설명하도록 유도하여 너무 빠르지 않도록 '강제'함으로써 잘못된 응답을 제공할 위험을 방지한다. LLM이 일반적인 일차 방정식을 풀게 하고 싶다고 가정하자. 이를 위해 LLM이 따라야 할 기본 추론 목록을 제공한다.

In [15]:
system_message = """
일반적인 1차 방정식을 해결하려면 다음 단계를 따르세요.

1. 방정식 식별 : 해결하려는 방정식을 식별합니다.
   방정식은 'ax + b = c' 형태여야 합니다.
   여기서 'a'는 변수의 계수, 'x'는 변수, 'b'는 상수, 'c'는 또 다른 상수입니다.
2. 변수 고립화 : 목표는 변수 'x'를 방정식 한쪽에 고립시키는 것입니다.
   이를 위해 다음 단계를 수행합니다:
   a. 상수 추가 또는 빼기 : 상수를 한쪽으로 이동시키기 위해 양쪽에서 'b'를 더하거나 뺍니다.
   b. 계수로 나누기 : 'x'를 고립시키기 위해 양쪽을 'a'로 나눕니다.
      'a'가 0이면 방정식에는 고유한 해가 없을 수 있습니다.
3. 단순화 : 방정식의 양쪽을 최대한 단순화합니다.
4. 'x'에 대해 해결 : 'x'를 한쪽에 고립시키면 해결책을 얻을 수 있습니다.
   이는 'x = 값' 형태가 될 것입니다. 
5. 해 검토 : 찾은 'x' 값을 원래 방정식에 대입하여 방정식을 만족하는지 확인합니다.
   그렇다면 올바른 해결책을 찾은 것입니다.
6. 해 표현 : 해결책을 명확하고 간결한 형태로 작성합니다.
7. 특수 경우 고려 : 'a'가 0일 때 해가 없거나 무한히 많은 해가 있을 수 있는 특수한 경우를 인지합니다.

방정식 : 
"""

equation = "3x + 5 = 11"

response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': equation},
    ]
)

In [16]:
print(response.choices[0].message.content)

주어진 방정식은 \(3x + 5 = 11\)입니다. 이를 해결하기 위해서 다음 단계를 따르겠습니다.

1. 방정식 식별: 방정식은 \(3x + 5 = 11\)입니다.

2. 변수 고립화: \(x\)를 한쪽에 고립시켜야 합니다.
   a. 상수 추가 또는 빼기: 양쪽에서 5를 빼줍니다.
   \[
   3x + 5 - 5 = 11 - 5
   \]
   \[
   3x = 6
   \]

   b. 계수로 나누기: 양쪽을 3으로 나눕니다.
   \[
   \frac{3x}{3} = \frac{6}{3}
   \]
   \[
   x = 2
   \]

3. 단순화: 이미 최대한 단순화 되었습니다.

4. 'x'에 대해 해결: \(x = 2\)로 얻었습니다.

5. 해 검토: \(x = 2\)를 원래 방정식에 대입하여 확인합니다.
   \[
   3(2) + 5 = 6 + 5 = 11
   \]
   원래 방정식을 만족하므로 올바른 해결책입니다.

6. 해 표현: 해결책은 \(x = 2\)입니다.

7. 특수 경우 고려: \(a\)가 0이 아닌 경우이므로 별도의 특별한 경우는 없습니다.

따라서 주어진 방정식 \(3x + 5 = 11\)의 해는 \(x = 2\)입니다.


모델은 meta prompt에 지정된 7단계를 명확하게 따랐으며, 이를 통해 이 과업을 수행하는 데 '시간적 여유'를 가질 수 있다. 응답하기 전에 추론이 필요한 더 복잡한 과업에서 더 나은 결과를 얻기 위해 이 기능을 단답형 prompt와 결합할 수도 있다.

CoT를 사용하면 모델에 중간 추론 단계를 생성하도록 유도한다. 이는 다음 절에서 살펴볼 또 다른 추론 기법의 구성 요소이기도 하다.

### 3. Reason and Act (ReAct)

Shunyu Yao 등의 논문 ReAct : Synergizing Reasoning and Acting in Language Models ReAct에 소개된 ReAct는 추론과 행동을 LLM과 결합하는 일반적 패러다임이다. ReAct는 언어 모델에 과업에 대한 언어적 추론 추적과 행동을 생성하도록 유도하고 웹 검색이나 DB와 같은 외부 소스에서 정보를 얻기도 한다. 이를 통해 언어 모델은 동적 추론을 수행하고, 외부 정보를 기반으로 행동 계획을 신속하게 조정할 수 있다. 예를 들어, 먼저 질문에 대해 추론한 다음 웹에 질의를 보내는 행동을 수행하고 검색 결과에서 정보를 수신한 다음 결론에 도달할 때까지 이러한 생각, 행동, 관찰 루프를 계속해 언어 모델에 질문에 대한 답변을 유도할 수 있다. CoT와 ReAct 접근 방식의 차이점은, CoT는 언어 모델에 과업에 대한 중간 추론 단계를 생성하라는 메세지를 표시하는 반면, ReAct는 언어 모델에 과업에 대한 중간 추론 단계, 행동 및 관찰을 생성하라는 메세지를 표시한다는 점이다.

'행동' 단계는 일반적으로 웹 검색과 같은 외부 도구와 상호 작용할 수 있는 LLM의 가능성과 관련이 있다. 예를 들어, 비교적 최근에 주식시장이 하락한 이유에 대한 정보를 모델에 요청하고 싶다고 가정하자. 이를 위해 SerpAPI 도구를 활용해 똑똑한 langchain Agent를 구축하고자 한다. 아래 예시를 보고, 어떤식으로 구현할 수 있는지 살펴보도록 한다. 아직 Agent 모델에 대해서는 진행하지 않았으므로, 어떤 식으로 진행이 되는지 간단히 살펴본다.

dependency : `langchain-community`, `serpapi`, `google-search-results`

In [18]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model_name = 'gpt-4o-mini',
    api_key = openai.api_key,
    
)
tools = load_tools(
    ['serpapi'],
    serpapi_api_key = serp_api_key
)
prompt = hub.pull('hwchase17/react')

agent = create_react_agent(chat, tools, prompt)
agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    verbose = True
)

/Users/junghunlee/anaconda3/envs/langchain/lib/python3.11/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


검색을 위해 langchain에서는 ReAct와 관련된 에이전트가 미리 생성되어 있다.

In [19]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


실제로 사용한 결과는 다음과 같다. input을 프롬프트 형식으로 유지한다면, 양방향 소통이 가능한 에이전트를 생성할 수 있다.

In [20]:
result = agent_executor.invoke(
    {'input': '2024년 8월 5일 미국의 주식시장이 갑자기 하락한 이유는?'}
)
print(result)



> Entering new AgentExecutor chain...
이 질문에 대한 답변을 제공하기 위해서는 2024년 8월 5일의 미국 주식시장 하락에 대한 구체적인 정보를 찾아야 합니다. 최근의 사건이나 경제 지표가 주식시장에 영향을 미쳤을 수 있으므로, 이를 조사하는 것이 중요합니다. 
Action: Search
Action Input: "2024년 8월 5일 미국 주식시장 하락 이유"['미국발 경기침체 우려와 주요국 통화정책 변화, 중동 분쟁과 미국 대통령 선거 등 대외적 불확실성이 해소되지 않은 까닭에 증권시장에선 당분간 변동성이 ...', '증권사들의 프로그램 매매와 미국 경제의 불안이 이유였다고 하지만 정확한 이유는 아닌 것으로 밝혀졌다. 블랙먼데이에 미국 시장 대형주인 IBM, 엑손( ...', '--> 블랙웰 출시지연 - AI서비스 개발 및 출시 지연 - AI 관심도 하락 - 투자 축소로 이어질 가능성도! ... 월 미국 금리인하로 엔 캐리 트레이드 청산은 ...', "미국 경기 둔화가 지표로 확인되며 경기 침체 진단에 증시가 폭락한 가운데 엔화 가치가 급등하며 '엔 캐리 트레이드' 청산 공포까지 커지고 있다. 4일 ...", '브래디 특별조사위원회는 블랙먼데이의 원인으로 미국의 재정적자, 국제수지 적자, 고(高)주가 현상, 금리 인상에 대한 불안감 등 복합적인 대내외 변수를 ...', '미국 경기 침체 우려, 외국인 투자자의 대량 매도, 엔 캐리 트레이드 청산 우려, 인공지능 관련 주식의 수익성 우려, 중동 지정학적 불안 등이 주요 원인 ...', '아시아 증시 급락은 미국 실업률 상승, 제조업 경기 전망 악화, AI(인공지능) 버블론 대두 등 미국 경기 침체 신호가 동시에 터져 나오고 있기 때문이다.', '이번 주(8월 5일~8월 9일) 뉴욕증시는 미국의 경기 침체 우려 속 높은 변동성을 보일 것으로 전망된다.지난주 뉴욕증시는 여러 대형 이벤트 속 롤러 ...']이 정보를 바탕으로 2024년 8월 5일의 미국 주식시장 하락 원인을 정리할

이제 더욱 세밀한 문제에 대해 질문하고 중간 단계를 살펴봄으로써 에이전트를 시험해 보자.

In [21]:
result = agent_executor.invoke(
    {'input': '2025년 1월 금리 인하가 되지 않을 것이란 우려가 확산되면서 시장의 변동성이 커졌는데, FOMC 이후에 주식 시장이 하락할까? 그에 대한 이유는?'}
)
print(result)



> Entering new AgentExecutor chain...
이번 질문은 2025년 1월 금리 인하와 FOMC(연방공개시장위원회) 회의 이후 주식 시장의 변동성에 대한 우려를 다루고 있다. 이를 분석하기 위해서는 현재 경제 상황, 금리 정책의 방향, 그리고 FOMC의 최근 결정들이 주식 시장에 미치는 영향을 살펴볼 필요가 있다. 따라서, 최근 경제 뉴스와 전문가의 분석을 찾아보는 것이 좋겠다.
Action: Search
Action Input: "2025년 금리 인하 FOMC 이후 주식 시장 전망"['12월 FOMC 점도표에 따르면, 연준은 2025년 금리 전망치를 3.9%로 상향 조정하며 금리 인하 횟수를 기존 4회에서 2회로 줄였습니다. 이는 2025년의 경제 ...', 'FOMC를 통해보는 2025년 금리 및 경기 전망! · 1. 정책금리를 25bp 인하하면서 올해를 총 100bp 인하로 마무리 (4.50%) · 1. 미국 10년물 국채금리 · 2. 원달러 환율 급등.', '기존 내년 말 전망 3.4%보다 높아졌다. 연준이 보는 내년 한 해 금리 인하 횟수 전망 기존 4차례에서 2차례로 감소한 것이다. 연준은 12월 연방공개시장 ...', '2025년 4분기 3%까지 가파른 금리인하를 기대했던 시장 전망은 10월에는 3.5% 수준으로 줄어들고 11월 들어서는 4.0% 수준으로 후 퇴한 상황이다.', '주식 ... 년 2회 금리인하 시사 이후 양호한 성장과 노동시장 여건, 트럼프 정부 출범 이후 인플레이션 재점화 우려 등으로 금리인하 주기 단축 전망 강화', '장기적으로는 자사의 계량 경제 모델에 따르면 미국 연방기금 금리가 2025년에 약 3.50%, 2026년에 3.00% 정도로 추세를 이룰 것으로 예상됩니다.', '금리인하 지연에 대한 불안심리 완화 예상. 현재 시장은 연준 점도표(2회)보다 매파적인 25년 금리 1회 인하를 선반영 중. 시. 장 컨센서스가 더 이상 ...', '금리 전망을 보여주는 점도표에서는 금리 인하 횟수의 중간값은 기

모델이 답을 도출하기까지 어떤 식으로 작동하였는지 주목하자. 이는 모델에 단계별로 생각하도록 유도하고, 추론의 각 단계를 명시적으로 정의하는 것이 어떻게 모델이 대답하기 전에 '더 현명하고' 더 주의하게 만드는지를 보여주는 좋은 예이다. 이는 또한 환각을 방지하는 훌륭한 기법이기도 하다.

전반적으로 prompt engineering은 강력한 분야로, 아직 초기 단계에 있지만 이미 LLM 기반의 애플리케이션에서 널리 채택되고 있다. 다음 장에서는 웹데이터를 수집해 투자자에게 유용한 정보를 제공하는 애플리케이션을 디자인 해 보도록 하자.